#  Distributed Computing with Big Data 

In [36]:
!pip install pyspark


In [34]:
from pyspark.sql import SparkSession

In [35]:
#Initialize a SparkContext to use PySpark.
# Create a Spark session
spark = SparkSession.builder.appName("PageRank").getOrCreate()

In [8]:
#Loading the data 
# Input data as an RDD of (source_URL, neighbor_URL) pairs
links = spark.sparkContext.parallelize([
    ("n1", "n2"), ("n2", "n3"), ("n3", "n4"), ("n4", "n5"), ("n5", "n1"),
    ("n1", "n4"), ("n1", "n5"), ("n2", "n1"), ("n2", "n5"), ("n3", "n1")
])

In [9]:
# Initialize the ranks of all URLs to 1.0
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [39]:
#PageRank Computation
#Implement PageRank Algorithm
def pagerank(nodes, num_iterations):
  node_page_ranks = nodes.mapValues(lambda x: 1.0)

  for i in range(num_iterations):
    contributions = nodes.join(node_page_ranks) \
      .flatMap(lambda x: [(neighbor, x[1][1] / len(x[1][0])) for neighbor in x[1][0]])

    node_page_ranks = contributions.reduceByKey(lambda x, y: x + y)

  return node_page_ranks

In [45]:
#Handling Dangling Nodes
def create_contributions(nodes):
  contributions = nodes.join(nodes) \
                      .flatMap(lambda x: [(neighbor, x[1][1] / len(x[1][0])) for neighbor in x[1][0]])

  return contributions

def pagerank(nodes, num_iterations):
  for i in range(num_iterations):
    contributions = create_contributions(nodes)

    node_page_ranks = contributions.reduceByKey(lambda x, y: x + y)

  return node_page_ranks

In [40]:
# Define the PageRank functions
def compute_contributions(url_neighbors, rank):
    num_neighbors = len(url_neighbors[1])
    for neighbor in url_neighbors[1]:
        yield (neighbor, rank / num_neighbors)

In [41]:
def recalculate_rank(contributions):
    return damping_factor + (1.0 - damping_factor) * contributions

In [42]:
# Define the damping factor and the number of iterations
damping_factor = 0.85
num_iterations = 10

In [47]:
print(ranks)
node_page_ranks = ranks
# Collect and print final PageRank values
final_page_ranks = node_page_ranks.collect()
for node, rank in final_page_ranks:
    print(f"{node}: {rank}")


PythonRDD[2] at RDD at PythonRDD.scala:53


n1: 1.0
n2: 1.0
n3: 1.0
n4: 1.0
n5: 1.0
n1: 1.0
n1: 1.0
n2: 1.0
n2: 1.0
n3: 1.0
